In [1]:

from langchain_community.llms import LlamaCpp


def load_llm():
    return LlamaCpp(
        model_path="models/llama-2-7b-chat.Q4_K_M.gguf",
        n_gpu_layers=40,
        n_batch=512,  # Batch size for model processing
        verbose=False,  # Enable detailed logging for debugging
    )

# Load the LlamaCpp language model, adjust GPU usage based on your hardware
llm = load_llm()

# Simple Chatbot

In [2]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define the prompt template with a placeholder for the question
template = """
Question: {question}

Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Create an LLMChain to manage interactions with the prompt and model
llm_chain = LLMChain(prompt=prompt, llm=llm)

'''while True:
    question = input("> ")
    answer = llm_chain.run(question)
    print(answer, '\n')'''

In [7]:

print("Chatbot initialized, ready to chat...")
question = 'how many soldiers did the sniper Lyudmila Pavlichenko kill?'  # 309, but Llama2 doesn't know this initially
answer = llm_chain.run(question)
print(answer, '\n')

Chatbot initialized, ready to chat...
Lyudmila Pavlichenko was a Soviet sniper during World War II. She is credited with killing 89 German soldiers during the war, making her one of the most successful female snipers in history.

Source:
www.history.com/topics/world-war-ii/lyudmila-pavlichenko 



# RAG Chatbot

In [8]:
from langchain.vectorstores import FAISS

In [9]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import CTransformers
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

llm = load_llm()

def create_vector_store(data_dir):
    '''Create a vector store from PDF files'''
    # define what documents to load
    loader = DirectoryLoader(path=data_dir, glob="*.pdf", loader_cls=PyPDFLoader)

    # interpret information in the documents
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                              chunk_overlap=50)
    texts = splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    # create the vector store database
    db = FAISS.from_documents(texts, embeddings)
    return db

db = create_vector_store('pdf_files')

In [10]:


def create_prompt_template():
    # prepare the template we will use when prompting the AI
    template = """Use the provided context to answer the user's question.
    If you don't know the answer, respond with "I do not know".

    Context: {context}
    Question: {question}
    Answer:
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=['context', 'question'])
    return prompt

def create_chain():
    #db = create_vector_store(data_dir='data')
    #llm = load_llm()
    prompt = create_prompt_template()
    retriever = db.as_retriever(search_kwargs={'k': 2})
    chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type='stuff',
                                        retriever=retriever,
                                        return_source_documents=False,
                                        chain_type_kwargs={'prompt': prompt})
    return chain


chain = create_chain()

def query_doc(chain, question):
    return chain({'query':question})['result']


def main():
  chain = create_chain()

  print("Chatbot for PDF files initialized, ready to query...")
  while True:
      question = input("> ")
      answer = query_doc(chain, question)
      print(': ', answer, '\n')

#main()

In [15]:
question = 'how many soldiers did the sniper Roza Shanina kill?'
answer = query_doc(chain, question)
print(': ', answer, '\n')

:   Roza Shanina killed 59 soldiers during World War II. 



# True / False Measuring

In [11]:
def create_tf_prompt_template():
    # prepare the template we will use when prompting the AI
    template = """Use the provided context to determine whether the statement is true or false.
    Just respond with "true" or "false".
    If you don't know the answer, respond with "I do not know".

    Context: {context}
    Statement: {statement}
    Answer:
    """

    prompt = PromptTemplate(
        template=template,
        input_variables=['context', 'statement'])
    return prompt

def create_chain():
    #db = create_vector_store(data_dir='data')
    #llm = load_llm()
    prompt = create_prompt_template()
    retriever = db.as_retriever(search_kwargs={'k': 2})
    chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type='stuff',
                                        retriever=retriever,
                                        return_source_documents=False,
                                        chain_type_kwargs={'prompt': prompt})
    return chain


chain = create_chain()

def query_doc(chain, statement):
    return chain({'query':statement})['result']

In [13]:
#statement = input("> ")
answer = query_doc(chain, "ragworms are not a kind of worm.")
print(': ', answer, '\n')
answer = query_doc(chain, "ragworms are a type of worm.")
print(': ', answer, '\n')


:   I do not know 

:   I do not know. 

